In [1]:
import carla
import random

In [2]:
client = carla.Client('localhost', 2000)
# client.load_world("Town05")
world = client.get_world()

In [3]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

print(location, rotation)

Location(x=-172.196579, y=183.859116, z=27.638052) Rotation(pitch=0.000000, yaw=-45.592148, roll=0.000000)


In [4]:
# Set the spectator to a position
# Intersection 1
spectator.set_transform(carla.Transform(carla.Location(x=-62.010975, y=1.288139, z=17.589510),carla.Rotation(pitch=-48.045647, yaw=53.329460, roll=0.000444)))
# Intersection 2
# spectator.set_transform(carla.Transform(carla.Location(x=123.813690, y=3.087291, z=18.886221),carla.Rotation(pitch=-27.611475, yaw=149.086655, roll=0.000053)))
# Intersection 3
# spectator.set_transform(carla.Transform(carla.Location(x=-62.647594, y=146.626755, z=9.927452),carla.Rotation(pitch=-20.047110, yaw=-45.562790, roll=0.000417)))
# Intersection 4
# spectator.set_transform(carla.Transform(carla.Location(x=-125.967690, y=-0.002219, z=12.125855),carla.Rotation(pitch=-22.860840, yaw=33.173634, roll=0.000728)))


Add Traffic

In [23]:
weather = carla.WeatherParameters(
    sun_altitude_angle=-90.0
)
world.set_weather(weather)


---------- RGB camera ----------

In [ ]:
camera_trans = carla.Transform(carla.Location(z=0.1))

camera_bp= world.get_blueprint_library().find('sensor.camera.rgb')

rgb_camera = world.spawn_actor(camera_bp, camera_trans, attach_to=spectator)

In [ ]:
rgb_camera.listen(lambda image: image.save_to_disk('rgb_output2/%06d.png' % image.frame))

In [ ]:
rgb_camera.stop()

---------- DVS camera ----------

In [ ]:
camera_trans = carla.Transform(carla.Location(z=0.1))

dvs_camera_bp= world.get_blueprint_library().find('sensor.camera.dvs')

dvs_camera = world.spawn_actor(dvs_camera_bp, camera_trans, attach_to=spectator)

In [ ]:
import csv
import numpy as np
import time
import cv2

In [ ]:
def dvs_callback(data, dvs_output_path): # store in csv file
    print("length = ",len(data))

    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
    with open(dvs_output_path, mode="a",  newline='') as file:
        writer = csv.writer(file)
        # dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        #     ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
        for event in dvs_events:
            writer.writerow(event)
        file.close()
    

In [ ]:
def dvs_callback_img(data): #store in image
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
    # data_dict['dvs_image'] = np.zeros((data.height, data.weight, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'],dvs_events[:]['x'],dvs_events[:]['pol']*2] = 255

    # data_dict['dvs_image'][:,:,0:3] = dvs_img
    # print(dvs_img)
    # dvs_img.save_to_disk('output/%06d.png' % dvs_img.frame)
    cv2.imshow("dvs_imh", dvs_img)
    cv2.imwrite(f'output/{data.frame}.png', dvs_img)
    
    

In [ ]:
dvs_output_path = "dvs_output.csv"
with open(dvs_output_path, mode="w",  newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['x', 'y', 't', 'pol'])
    file.close()

In [ ]:
dvs_camera.stop()

In [ ]:
dvs_camera.listen(lambda DVSEventArray: dvs_callback_img(DVSEventArray))
# time.sleep(3)
# dvs_camera.stop()

In [ ]:
dvs_camera.listen(lambda DVSEventArray: dvs_callback(DVSEventArray, dvs_output_path))
time.sleep(3)
dvs_camera.stop()

In [ ]:

dvs_output_path = "dvs_output.csv"

with open(dvs_output_path, mode="r",  newline='') as file:
    first = 0
    second = 0
    reader = csv.reader(file)
    for row in reader:
        second = int(row[2])
        if first > second:
            print("ERROR!!!")
            print(first, second)
        first = int(row[2])
    file.close()